In [137]:
import dask
import dask.dataframe as dd
from dask.distributed import Client, wait, LocalCluster
import dask_ml
import dask.array as da
from dask_ml.model_selection import train_test_split
from scipy.stats import uniform, loguniform
from dask_ml.wrappers import ParallelPostFit
from dask_ml.model_selection import HyperbandSearchCV, GridSearchCV

In [10]:
client = Client(LocalCluster())
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 7.87 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60851,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 7.87 GiB
Comm: tcp://127.0.0.1:60887,Total threads: 1
Dashboard: http://127.0.0.1:60888/status,Memory: 1.97 GiB
Nanny: tcp://127.0.0.1:60856,


In [64]:
df = dd.read_csv("../data/day.csv").drop("dteday", axis = 1)
df

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
npartitions=1,,,,,,,,,,,,,,,
,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [65]:
def take_samples(df, frac):
    return df.sample(frac = frac)

fut = []  
for i in range(5):
    samples = client.submit(take_samples, df, 0.1) 
    fut.append(samples) 
    wait(fut)  

In [66]:
samples = []
for f in fut:
    samples.append(f.result())
sample = samples[0]

In [67]:
sample.compute()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
655,656,4,1,10,0,3,1,1,0.455833,0.450108,0.692917,0.101371,979,6482,7461
578,579,3,1,8,0,3,1,1,0.717500,0.667308,0.677500,0.141179,1074,6506,7580
637,638,4,1,9,0,6,0,1,0.542500,0.530296,0.542917,0.227604,2589,5966,8555
715,716,4,1,12,0,0,0,2,0.362500,0.369938,0.838750,0.100742,538,3248,3786
43,44,1,0,2,0,0,0,1,0.316522,0.324113,0.457391,0.260883,397,1192,1589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,365,1,0,12,0,6,0,1,0.410000,0.414121,0.615833,0.220154,665,1820,2485
174,175,3,0,6,0,5,1,1,0.724167,0.656583,0.573333,0.222025,969,4022,4991
521,522,2,1,6,0,2,1,2,0.540833,0.525892,0.613333,0.209575,968,6033,7001
525,526,2,1,6,0,6,0,1,0.710833,0.648367,0.437083,0.144287,2708,4790,7498


In [68]:
sample.groupby("season").apply(lambda x: x["temp"].mean()).compute()

`meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result


season
1    0.308133
2    0.538553
3    0.743650
4    0.445375
dtype: float64

In [69]:
train, test = sample.random_split([0.8, 0.2])
X = train.drop("temp", axis = 1)
y = train["temp"]

Cluster

In [72]:
from  dask_ml.cluster import KMeans
cluster = KMeans(n_clusters=2).fit(train.to_dask_array(lengths = True))

Incremental learning

In [100]:
from dask_ml.wrappers import Incremental
from sklearn.linear_model import SGDRegressor, SGDClassifier

estimator = SGDRegressor(random_state=10, max_iter=100)
reg = Incremental(estimator)
reg.fit(X, y)
reg.estimator_.score(X, y)

-0.25571081601792933

Naive Bayes

In [90]:
from dask_ml import datasets
from dask_ml.naive_bayes import GaussianNB

X, y = datasets.make_classification(chunks=50)
gnb = GaussianNB()
gnb.fit(X, y)
gnb.predict_proba(X) #.compute()

dask.array<exp, shape=(100, 2), dtype=float64, chunksize=(50, 1), chunktype=numpy.ndarray>

Pipeline

In [95]:
from sklearn.pipeline import Pipeline  # regular scikit-learn pipeline
from dask_ml.cluster import KMeans
from dask_ml.decomposition import PCA

estimators = [('reduce_dim', PCA()), ('nb', GaussianNB())]
pipe = Pipeline(estimators)
pipe.fit(X, y)
pipe.predict_proba(X) #.compute()

dask.array<exp, shape=(100, 2), dtype=float64, chunksize=(50, 1), chunktype=numpy.ndarray>

Cross Validation and Hyper Parameter Search

In [141]:
from dask_ml.linear_model import LogisticRegression

X, y = datasets.make_classification(chunks=20)
X_train, X_test, y_train, y_test = train_test_split(X, y)
params = {
    "penalty": ["l2", "l1"],
    "fit_intercept": [True, False]
}

est = ParallelPostFit(LogisticRegression())
est = LogisticRegression()
search = HyperbandSearchCV(est, params, random_state=0)

search.fit(X_train, y_train, classes=[0, 1]);
search.score(X_test, y_test)

KeyboardInterrupt: 